#### iris 데이터셋 활용해서 꽃잎 너비 예측 모델
 - 데이터셋 : iris.csv에서 2개의 Feature 사용
 - 구현프레임워크 : Pytorch

- [1] 모듈 로딩 및 데이터 준비

In [17]:
# 모듈 로딩
import torch                                    # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 손실, 거래 등 함수 관련 모듈 
import torch.optim as optimizer                 # 최적화 기법 관련 모듈
from torchmetrics.regression import R2Score     # 성능지표 관련 모듈   - 추가 설치
from torchinfo import summary                   # 모델 정보 관련 모듈  - 추가 설치

import pandas as pd
from sklearn.model_selection import  train_test_split


In [18]:
# 모델의 가중치 및 절편 값 고정 설정
torch.manual_seed(1)

# 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'DEVICE => {DEVICE}')

DEVICE => cpu


In [19]:
# [1-1] 데이터 로딩
irisDF = pd.read_csv(r'C:\Users\KDP-17\EX_PANDAS6\TORCH_DL\data\iris.csv',usecols=[0,1,2,3])
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


[2] 모델 준비
 - 학습방법 : 지도학습 > 회귀
 - 알고리즘 : 선형관계 >> 선형모델 ==> nn.Linear

In [20]:
# 모델 설계
# 입력층에 입력값(feature)  => sepal.length, sepal.width, petal.length 3개
# 출력층에 출력값(target) => petal.width 1개
# 입력층 : 입력 피쳐 3개, 출력 입력층에 존재하는 퍼셉트론 개수 10개, AF - ReLU
#                                                      ㅣ     
#            -------------------------------------------
#           ㅣ
# 은닉층 : 입력 10개, 출력 입력층에 존재하는 퍼셉트론 개수 5개, AF - ReLU
#                                                       ㅣ
#                ----------------------------------------
#               ㅣ
# 출력층 : 입력 5개, 출력 타겟/라벨 개수 : 1개, AF - None

model = nn.Sequential(nn.Linear(3,10),
                      nn.ReLU(),
                      nn.Linear(10,5),
                      nn.ReLU(),
                      nn.Linear(5,1))

'''
입력층의 입력은 피쳐 개수로 고정
출력층의 출력은 타겟 개수로 고정
'''


'\n입력층의 입력은 피쳐 개수로 고정\n출력층의 출력은 타겟 개수로 고정\n'

In [21]:
# 모델 구조 확인
'''
Param 개개수는 (입력 개수 + 1) * 퍼셉트론 개수
'''
print(model)
summary(model,input_size=(2000,3))


Sequential(
  (0): Linear(in_features=3, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [2000, 1]                 --
├─Linear: 1-1                            [2000, 10]                40
├─ReLU: 1-2                              [2000, 10]                --
├─Linear: 1-3                            [2000, 5]                 55
├─ReLU: 1-4                              [2000, 5]                 --
├─Linear: 1-5                            [2000, 1]                 6
Total params: 101
Trainable params: 101
Non-trainable params: 0
Total mult-adds (M): 0.20
Input size (MB): 0.02
Forward/backward pass size (MB): 0.26
Params size (MB): 0.00
Estimated Total Size (MB): 0.28

In [22]:
# 가중치와 절편 확인
for name, name_parameter in model.named_parameters() :
    print(f'[{name}], [{name_parameter}],"\n"')

[0.weight], [Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119],
        [ 0.2710, -0.5435,  0.3462],
        [-0.1188,  0.2937,  0.0803],
        [-0.0707,  0.1601,  0.0285],
        [ 0.2109, -0.2250, -0.0421],
        [-0.0520,  0.0837, -0.0023],
        [ 0.5047,  0.1797, -0.2150],
        [-0.3487, -0.0968, -0.2490],
        [-0.1850,  0.0276,  0.3442],
        [ 0.3138, -0.5644,  0.3579]], requires_grad=True)],"
"
[0.bias], [Parameter containing:
tensor([ 0.1613,  0.5476,  0.3811, -0.5260, -0.5489, -0.2785,  0.5070, -0.0962,
         0.2471, -0.2683], requires_grad=True)],"
"
[2.weight], [Parameter containing:
tensor([[ 0.3103, -0.1338,  0.2371,  0.0037, -0.1666,  0.1625, -0.1679,  0.0930,
         -0.0913, -0.0347],
        [-0.3040, -0.1508,  0.1716, -0.0769,  0.3150,  0.2535, -0.0148, -0.2111,
          0.1926,  0.0981],
        [-0.2044,  0.2054,  0.1920,  0.2805, -0.1773, -0.0521, -0.0061,  0.0462,
         -0.2400, -0.2244],
        [ 0.1720, -0.0742,  0.1545,  0.018

[3] 최적화 인스턴스 준비

In [23]:
### 모델의 가중치와 절편 최적화 이후 인스턴스에 전달
adam_optimizer = optimizer.Adam(model.parameters(), lr=0.1)

[4] 학습 진행  ==> 개발자가 구현

 - [4-1] 데이터셋 Tensor화 진행 : 데이터 준비 시 진행 or 학습 전 진행

In [24]:
irisDF.columns[:-1]

Index(['sepal.length', 'sepal.width', 'petal.length'], dtype='object')

In [25]:
# 피쳐와 타겟 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[['petal.width']]

print(f'featureDF.shape : {featureDF.shape}, targetDF.shape : {targetDF.shape}')

featureDF.shape : (150, 3), targetDF.shape : (150, 1)


In [26]:
# Train, Test
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF,
                                                   test_size = 0.2,
                                                   random_state=5)

# Train, Valid
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                   test_size = 0.2,
                                                   random_state=5)

- [4-2] 학습진행 : 
   * 학습횟수 결정 ==> 에포크 설정
   * 배치 크기 결정
   * 배치 개수 계산

In [27]:
EPOCH = 1                                   # 처음 ~ 끝까지 공부하는 횟수
BATCH_SIZE = 12                             # 1에포크에서 한번 학습할 분량 크기

BATCH_CNT = X_train.shape[0]//BATCH_SIZE    # 1에포크에서 총 학습 횟수, 가중치 업데이트 횟수

print(f'EPOCH:{EPOCH}  BATCH_SIZE : {BATCH_SIZE}  BATCH_CNT : {BATCH_CNT}')

EPOCH:1  BATCH_SIZE : 12  BATCH_CNT : 8


In [28]:
# 모델 테스트/검증 함수 
# ==> 가중치,절편 업데이트 X 그래서 최적화 미진행 해야함
# ==> 현재 가중치와 절편으로 테스트 진행
def testing(testDF, targetDF, kind = 'Val') :

    # Tensor 화
    testTS = torch.FloatTensor(testDF.values).to(DEVICE)
    targetTS = torch.FloatTensor(targetDF.values).to(DEVICE)
    with torch.no_grad() :      # 가중치 및 절편 업데이트 X
        # (1) 학습진행 forward
        pre_y = model(testTS)

        # (2) 오차계산 - 손실함수
        loss = F.mse_loss(pre_y, targetTS)

        # (3) 성능평가 - R2
        r2 = R2Score()(pre_y,targetTS)
       
        # (4) 학습 과정 출력
        print(f'[{kind}] LOSS : {loss} R2 : {r2}   ')

    return loss, r2


In [29]:
# 모델 학습 함수
def training(featureTS, targetTS, valTS, valTargetTS) :
    #[[],[]] <= [train, val]
    loss_history = [[],[]]
    r2_history = [[],[]]

    for epoch in range(EPOCH) : 
        # 배치 손실 저장 변수
        bs_loss,bs_r2 = 0,0
        # 배치 크기만큼 학습 진행
        for i in range(BATCH_CNT) :
            start = i*BATCH_SIZE
            end = start + BATCH_SIZE
            
            
            # BATCH_SIZE 크기만큼만 데이터 추출해서 Tensor화 진행
            BSX_train = torch.FloatTensor(X_train[start:end].values).to(DEVICE)
            BSy_train = torch.FloatTensor(y_train[start:end].values).to(DEVICE)
            print(BSX_train.shape, BSX_train.device, BSX_train.dtype)
            print(BSy_train.shape, BSy_train.device, BSy_train.dtype)
            print()

        # (1) 학습진행 forward
        pre_y = model(BSX_train)

        # (2) 오차계산 - 손실함수
        loss = F.mse_loss(pre_y, BSy_train)
        bs_loss += loss.item()
        bs_r2 += R2Score()(pre_y,BSy_train).item()
       
        # (3) 최적화 - 가중치, 절편 업데이트
        adam_optimizer.zero_grad()
        loss.backward()
        adam_optimizer.step()

    val_loss, val_r2 = testing(valTS,valTargetTS)
    loss_history[1].append(val_loss.item())
    r2_history[1].append(val_r2.item())

    # 에포크 단위 손실과 성능지표(r2)
    loss_history[0].append(bs_loss/BATCH_CNT)
    r2_history[0].append(bs_r2/BATCH_CNT)


    # (4) 학습 과정 출력
    print(f'[{epoch}/{EPOCH}] TRAIN_LOSS : {loss_history[0]}  R2 : {r2_history[0][-1]}')
    print(f'VALID LOSS : {loss_history[1][-1]}  R2 : {r2_history[1][-1]}')

    return loss_history, r2_history 

In [30]:
# 모델 학습 진행
training(X_train,y_train,X_val,y_val)

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

torch.Size([12, 3]) cpu torch.float32
torch.Size([12, 1]) cpu torch.float32

[Val] LOSS : 0.5848502516746521 R2 : -0.4013630151748657   
[0/1] TRAIN_LOSS : [0.11050643771886826]  R2 : -0.19764670729637146
VALID LOSS : 0.5848502516746521  R2 : -0.4013630151748657


([[0.11050643771886826], [0.5848502516746521]],
 [[-0.19764670729637146], [-0.4013630151748657]])

In [31]:
# # 학습 후 loss 시각화
# import matplotlib.pyplot as plt

# plt.plot(range(1,101),loss[0], label='Train')
# plt.plot(range(1,101),loss[1], label=-'Val')
# plt.title('Train & Valid')
# plt.grid()
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

NameError: name 'loss' is not defined